In [40]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\alvar\AppData\Local\Temp\ipykernel_5712\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [42]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\alvar\Downloads\kg_train.csv", encoding="latin-1")

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [43]:
from sklearn.model_selection import train_test_split

# Split data: 80% training, 20% testing
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Print shapes to confirm the split
print("Training set shape:", train_data.shape)
print("Test set shape:", test_data.shape)

Training set shape: (800, 2)
Test set shape: (200, 2)


## Data Preprocessing

In [44]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [46]:
from bs4 import BeautifulSoup
import re

def clean_html(text):
    # Parse HTML
    soup = BeautifulSoup(text, "html.parser")

    # Remove JavaScript & CSS
    for script in soup(["script", "style"]):
        script.extract()  # Removes the script/style tag and content

    # Remove HTML comments
    cleaned_text = re.sub(r"<!--.*?-->", "", str(soup), flags=re.DOTALL)

    # Remove remaining HTML tags
    cleaned_text = BeautifulSoup(cleaned_text, "html.parser").get_text()

    return cleaned_text.strip()

# Apply cleaning to the dataset
train_data["text"] = train_data["text"].apply(clean_html)
test_data["text"] = test_data["text"].apply(clean_html)

# Show sample cleaned text
print(train_data["text"].head())

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836    BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
Name: text, dtype: object


C:\Users\alvar\AppData\Local\Temp\ipykernel_5712\2726123964.py:6: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, "html.parser")
C:\Users\alvar\AppData\Local\Temp\ipykernel_5712\2726123964.py:1

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [47]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\b\w\b', ' ', text)  # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Remove single characters from start
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'^b\s+', '', text)  # Remove prefixed 'b'
    
    return text

# Apply cleaning to the dataset
train_data["text"] = train_data["text"].apply(clean_text)
test_data["text"] = test_data["text"].apply(clean_text)

# Show sample cleaned text
print(train_data["text"].head())

29     regards mr nelson smithkindly reply me on my p...
535    have not been able to reach oscar this am we a...
695    huma abedin bi checking with pat on the k will...
557       can have it announced here on monday can today
836    bank of africaagence san pedro bp san pedro co...
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [48]:
import nltk
from nltk.corpus import stopwords

# Download stopwords (only needs to be run once)
nltk.download('stopwords')

# Set of English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply to datasets
train_data['text'] = train_data['text'].apply(remove_stopwords)
test_data['text'] = test_data['text'].apply(remove_stopwords)

# Show sample cleaned data
print(train_data['text'].head())


29     regards mr nelson smithkindly reply private em...
535           able reach oscar supposed send pdb receive
695    huma abedin bi checking pat k work jack jake r...
557                               announced monday today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [49]:
import nltk
from nltk.stem import WordNetLemmatizer

# Download WordNet data (only once)
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function for lemmatizing text
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply to datasets
train_data['text'] = train_data['text'].apply(lemmatize_text)
test_data['text'] = test_data['text'].apply(lemmatize_text)

# Print sample of cleaned and lemmatized text
print(train_data['text'].head())


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alvar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alvar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


29     regard mr nelson smithkindly reply private ema...
535           able reach oscar supposed send pdb receive
695    huma abedin bi checking pat k work jack jake r...
557                               announced monday today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [50]:
from collections import Counter

# Separate ham and spam messages
ham_messages = train_data[train_data['label'] == 0]['text']
spam_messages = train_data[train_data['label'] == 1]['text']

# Combine all words into one list for each
all_ham_words = ' '.join(ham_messages).split()
all_spam_words = ' '.join(spam_messages).split()

# Count word frequencies
ham_word_counts = Counter(all_ham_words)
spam_word_counts = Counter(all_spam_words)

# Top 10 words
top_10_ham = ham_word_counts.most_common(10)
top_10_spam = spam_word_counts.most_common(10)

# Print results
print("Top 10 Ham Words:\n", top_10_ham)
print("\nTop 10 Spam Words:\n", top_10_spam)

Top 10 Ham Words:
 [('would', 93), ('state', 92), ('pm', 88), ('president', 84), ('percent', 76), ('obama', 74), ('call', 73), ('secretary', 71), ('time', 70), ('mr', 70)]

Top 10 Spam Words:
 [('money', 799), ('account', 679), ('bank', 607), ('fund', 569), ('u', 436), ('business', 399), ('transaction', 350), ('country', 342), ('transfer', 329), ('company', 319)]


## Extra features

In [55]:
# Rename train/test data to match exercise format
data_train = train_data.copy()
data_val = test_data.copy()

# Assuming 'text' column is the preprocessed one, we rename it for clarity
data_train['preprocessed_text'] = data_train['text']
data_val['preprocessed_text'] = data_val['text']

# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
29,regard mr nelson smithkindly reply private ema...,1,regard mr nelson smithkindly reply private ema...,1,0,75
535,able reach oscar supposed send pdb receive,0,able reach oscar supposed send pdb receive,1,0,42
695,huma abedin bi checking pat k work jack jake r...,0,huma abedin bi checking pat k work jack jake r...,1,0,80
557,announced monday today,0,announced monday today,1,0,22
836,bank africaagence san pedro bp san pedro cote ...,1,bank africaagence san pedro bp san pedro cote ...,1,1,1063


## How would work the Bag of Words with Count Vectorizer concept?

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize vectorizer
vectorizer = CountVectorizer()

# Fit on training data and transform
X_train = vectorizer.fit_transform(data_train['preprocessed_text'])

# Transform validation/test data
X_val = vectorizer.transform(data_val['preprocessed_text'])

# Shape of the BoW matrices (samples x vocabulary size)
print(X_train.shape)
print(X_val.shape)


(800, 17235)
(200, 17235)


In [53]:
from scipy.sparse import hstack

# Select additional features
extra_features_train = data_train[['money_mark', 'suspicious_words', 'text_len']]
extra_features_val = data_val[['money_mark', 'suspicious_words', 'text_len']]

# Combine BoW and extra features
X_train_full = hstack([X_train, extra_features_train])
X_val_full = hstack([X_val, extra_features_val])

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_tfidf = vectorizer.transform(data_val['preprocessed_text'])

print("TF-IDF shape:", X_tfidf.shape)

TF-IDF shape: (5964, 70194)


In [66]:
from scipy.sparse import hstack

# Select extra features
extra_features_train = data_train[['money_mark', 'suspicious_words', 'text_len']]
extra_features_val = data_val[['money_mark', 'suspicious_words', 'text_len']]

# Combine
X_train_final = hstack([X_train_tfidf, extra_features_train])
X_val_final = hstack([X_val_tfidf, extra_features_val])

print("TF-IDF shape:", X_tfidf.shape)

TF-IDF shape: (5964, 70194)


## And the Train a Classifier?

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_train = data_train['label']
y_val = data_val['label']

# Initialize and train the model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_final, y_train)

# Predict on validation set
y_pred = model.predict(X_val_final)

# Evaluate
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))


Accuracy: 0.93

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.94       125
           1       0.92      0.89      0.91        75

    accuracy                           0.93       200
   macro avg       0.93      0.92      0.92       200
weighted avg       0.93      0.93      0.93       200


Confusion Matrix:
 [[119   6]
 [  8  67]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Multinomial Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Linear SVM": LinearSVC(max_iter=1000, random_state=42)
}

for model_name, model in models.items():
    print(f"\n{'='*30}\nTraining: {model_name}\n{'='*30}")
    
    # Train
    model.fit(X_train_final, y_train)
    
    # Predict
    y_pred = model.predict(X_val_final)
    
    # Evaluate
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", classification_report(y_val, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))


Training: Logistic Regression
Accuracy: 0.9300

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.94       125
           1       0.92      0.89      0.91        75

    accuracy                           0.93       200
   macro avg       0.93      0.92      0.92       200
weighted avg       0.93      0.93      0.93       200

Confusion Matrix:
 [[119   6]
 [  8  67]]

Training: Multinomial Naive Bayes
Accuracy: 0.7600

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.62      0.76       125
           1       0.61      0.99      0.76        75

    accuracy                           0.76       200
   macro avg       0.80      0.81      0.76       200
weighted avg       0.85      0.76      0.76       200

Confusion Matrix:
 [[78 47]
 [ 1 74]]

Training: Random Forest
Accuracy: 0.9550

Classification Report:
               precision    recall  f1-score   sup

c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
